# Example of correctly formatting data

For use in decoding (see "Examples_decoders_hc" and "Examples_kf_decoder_hc"), we need the following format of inputs:
- Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin
- The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"

In this example, we load Matlab data that contains 
- The spike times of all neurons. In Matlab, "spike_times" is a cell of size "number of neurons" x 1. Within spike_times{i} is a vector containing all the spike times of neuron i.
- A continuous stream of the output variables. In this example, we are aiming to decode position. In Matlab, "pos" is a matrix of size "number of recorded time points" x 2 (x and y positions were recorded) that contains the x and y position components at all time points. Time points that were not recorded have NaN values. "pos_times" is a vector that states the time at all recorded time points. 

We will put this data in the format described above, with the help of the functions "bin_spikes" and "bin_output" that are in the file "preprocessing_funcs.py"




## Import packages and functions
Note that you may need to specify the path below

In [1]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
###Import standard packages###
import numpy as np
from scipy import io
import sys

###Import functions for binning data for preprocessing###
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output

## Load Data
The data for this example can be downloaded at this [link](https://www.dropbox.com/s/94dhsgnx2cfs3jx/hc_data_raw.mat?dl=0)

It is the hc-2 dataset from [crcns](https://crcns.org/data-sets/hc/hc-2). Specifically, we use the dataset "ec014.333" 

In [14]:
###Load Hippo Example Data###

folder = '/Users/rbhatt6/Documents/MATLAB/'
# For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 
data=io.loadmat(folder+'hc_data_raw.mat')
spike_times=data['spike_times'] #Load spike times of all neurons
pos=data['pos'] #Load x and y positions
pos_times=data['pos_times'][0] #Load times at which positions were recorded

In [18]:
###Load Acc/Dec Data###

folder = '/Users/rbhatt6/Documents/MATLAB/'
# For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

# Decoder Velocity Outputs
sortIn = io.loadmat(folder+'cleanedSortIn.mat')
sortOutX = io.loadmat(folder+'sortOutX.mat')
sortOutY = io.loadmat(folder+'sortOutY.mat')

sortIn = sortIn['cleanedSortIn']
sortOutX = sortOutX['sortOutX']
sortOutY = sortOutY['sortOutY']

sortIn = np.squeeze(sortIn)
sortOutX = np.squeeze(sortOutX)
sortOutY = np.squeeze(sortOutY)

# Decoder Position Outputs
sortOutX_Pos = io.loadmat(folder+'sortOutXPos.mat')
sortOutY_Pos = io.loadmat(folder+'sortOutYPos.mat')

sortOutX_Pos = sortOutX_Pos['sortOutX']
sortOutY_Pos = sortOutY_Pos['sortOutY']

sortOutX_Pos = np.squeeze(sortOutX_Pos)
sortOutY_Pos = np.squeeze(sortOutY_Pos)


## User Inputs

In [9]:
dt=.2 #Size of time bins (in seconds)
t_start=pos_times[0] #Time to start extracting data - here the first time position was recorded
t_end=5608 #pos_times[-1] #Time to finish extracting data - when looking through the dataset, the final position was recorded around t=5609, but the final spikes were recorded around t=5608
downsample_factor=1 #Downsampling of output (to make binning go faster). 1 means no downsampling.

## Put data in binned format

In [10]:
#When loading the Matlab cell "spike_times", Python puts it in a format with an extra unnecessary dimension
#First, we will put spike_times in a cleaner format: an array of arrays
spike_times=np.squeeze(spike_times)
for i in range(spike_times.shape[0]):
    spike_times[i]=np.squeeze(spike_times[i])

# spike_times is still dtype = object!

In [11]:
###Preprocessing to put spikes and output in bins###

#Bin neural data using "bin_spikes" function
neural_data=bin_spikes(spike_times,dt,t_start,t_end)

#Bin output (position) data using "bin_output" function
pos_binned=bin_output(pos,pos_times,dt,t_start,t_end,downsample_factor)

## Save Data

In [17]:
import pickle

data_folder='/Users/rbhatt6/Documents/MATLAB/' #FOLDER YOU WANT TO SAVE THE DATA TO

with open(data_folder+'example_data_hc.pickle','wb') as f:
    pickle.dump([neural_data,pos_binned],f)

with open(data_folder+'sortInAccDec.pickle','wb') as f:
    pickle.dump(sortIn,f)

with open(data_folder+'sortOutXAccDec.pickle','wb') as f:
    pickle.dump(sortOutX,f)

with open(data_folder+'sortOutYAccDec.pickle','wb') as f:
    pickle.dump(sortOutY,f)

with open(data_folder+'sortOutXAccDecPos.pickle','wb') as f:
    pickle.dump(sortOutX_Pos,f)

with open(data_folder+'sortOutYAccDecPos.pickle','wb') as f:
    pickle.dump(sortOutY_Pos,f)
